In [ ]:
# --- First cell for all notebooks: works locally and in Google Colab ---

import os
import sys

# Detect if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Mount Google Drive in Colab
    from google.colab import drive
    drive.mount('/content/drive')
    
    # BASE_PATH: repo folder name after cloning in Colab
    BASE_PATH = "/content/Real-Time-Anomaly-Detection-in-IoMT-AD-Project"
else:
    # Local environment: BASE_PATH = project root (one level up from notebooks)
    BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))

print(f"📁 BASE_PATH detected as: {BASE_PATH}")

# Add Spark and Models folders to Python path for imports
sys.path.append(os.path.join(BASE_PATH, "Spark"))
sys.path.append(os.path.join(BASE_PATH, "Models"))
print("✅ PYTHONPATH updated. Spark and Models folders included.")


Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, udf
from pyspark.sql.types import StructType, StructField, FloatType, StringType
import json
import numpy as np
from tensorflow.keras.models import load_model
import joblib


# Load scaler + feature maps + threshold

In [ ]:
# Load scaler
scaler = joblib.load(os.path.join(BASE_PATH, "Models/scaler.pkl"))
print("Scaler loaded ✅")

# Load selected features
with open(os.path.join(BASE_PATH, "Spark/selected_features.json")) as f:
    selected_features = json.load(f)

# Load all features
with open(os.path.join(BASE_PATH, "Dataset/intermediate/features.json")) as f:
    all_features = json.load(f)

# Load threshold
with open(os.path.join(BASE_PATH, "Models/threshold.json")) as f:
    threshold = json.load(f)["threshold"]

# Load LSTM autoencoder
model = load_model(os.path.join(BASE_PATH, "Models/lstm_autoencoder.h5"), compile=False)
print("LSTM autoencoder loaded ✅")


Scaler loaded OK!


# Start Spark Session with Kafka

In [ ]:
spark = (SparkSession.builder
         .appName("IoMT_Anomaly_Detector")
         .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1")
         .getOrCreate())
spark.sparkContext.setLogLevel("ERROR")
print("Spark session started ✅")


# Define schema for incoming JSON from Kafka

In [8]:
from pyspark.sql.types import StructType, StructField, FloatType, StringType

schema = StructType([
    StructField("flow_duration", FloatType()),
    StructField("Header_Length", FloatType()),
    StructField("Protocol Type", FloatType()),
    StructField("Duration", FloatType()),
    StructField("Rate", FloatType()),
    StructField("Srate", FloatType()),
    StructField("Drate", FloatType()),
    StructField("fin_flag_number", FloatType()),
    StructField("syn_flag_number", FloatType()),
    StructField("rst_flag_number", FloatType()),
    StructField("psh_flag_number", FloatType()),
    StructField("ack_flag_number", FloatType()),
    StructField("ece_flag_number", FloatType()),
    StructField("cwr_flag_number", FloatType()),
    StructField("ack_count", FloatType()),
    StructField("syn_count", FloatType()),
    StructField("fin_count", FloatType()),
    StructField("urg_count", FloatType()),
    StructField("rst_count", FloatType()),
    StructField("HTTP", FloatType()),
    StructField("HTTPS", FloatType()),
    StructField("DNS", FloatType()),
    StructField("Telnet", FloatType()),
    StructField("SMTP", FloatType()),
    StructField("SSH", FloatType()),
    StructField("IRC", FloatType()),
    StructField("TCP", FloatType()),
    StructField("UDP", FloatType()),
    StructField("DHCP", FloatType()),
    StructField("ARP", FloatType()),
    StructField("ICMP", FloatType()),
    StructField("IPv", FloatType()),
    StructField("LLC", FloatType()),
    StructField("Tot sum", FloatType()),
    StructField("Min", FloatType()),
    StructField("Max", FloatType()),
    StructField("AVG", FloatType()),
    StructField("Std", FloatType()),
    StructField("Tot size", FloatType()),
    StructField("IAT", FloatType()),
    StructField("Number", FloatType()),
    StructField("Magnitue", FloatType()),
    StructField("Radius", FloatType()),
    StructField("Covariance", FloatType()),
    StructField("Variance", FloatType()),
    StructField("Weight", FloatType()),
    StructField("label", StringType())
])


# Subscribe to Kafka

In [ ]:
df = (spark.readStream
      .format("kafka")
      .option("kafka.bootstrap.servers", "localhost:9092")
      .option("subscribe", "iomt_traffic_stream")
      .load())

stream_df = df.selectExpr("CAST(value AS STRING)")


# Parse JSON to columns

In [ ]:
json_df = stream_df.select(from_json(col("value"), schema).alias("data")).select("data.*")


# Convert incoming row → Numpy → scale → select → run model

In [ ]:
def infer(*cols):
    x = np.array(cols, dtype=np.float32).reshape(1, -1)
    x_scaled = scaler.transform(x)
    idx = [all_features.index(f) for f in selected_features]
    x_sel = x_scaled[:, idx]
    x_lstm = np.expand_dims(x_sel, axis=1)
    recon = model.predict(x_lstm, verbose=0)[0, 0]
    err = float(np.mean((x_sel - recon)**2))
    anomaly = 1 if err > threshold else 0
    return anomaly, err

infer_udf = udf(infer, returnType=StructType([
    StructField("anomaly", StringType()),
    StructField("error", FloatType())
]))


# Apply the UDF

In [ ]:
feature_columns = [col(f).cast("float") for f in all_features]  # only numeric
result_df = json_df.withColumn("prediction", infer_udf(*feature_columns))


# Output anomalies to console / DB / dashboard

In [ ]:
query = (result_df
          .select("flow_duration", "prediction.*")
          .writeStream
          .outputMode("append")
          .format("console")
          .option("truncate", False)
          .start())


In [ ]:
query.awaitTermination()


StreamingQueryException: [STREAM_FAILED] Query [id = 109a8ea6-dcf2-458a-b498-af5f81b333a3, runId = a3ee6ab7-36af-44f3-b734-e7244aa4996d] terminated with exception: org.apache.kafka.common.errors.TimeoutException: Timed out waiting for a node assignment. Call: describeTopics